# AI Document Search Assistant
## Features
- Parses documents from `.txt`, `.pdf`, and `.docx`
- Converts them into embeddings
- Stores them in ChromaDB
- Queries with LangChain integration

In [68]:
# 📦 Install necessary packages
#!pip install -q langchain chromadb python-docx PyMuPDF
#!pip install -U langchain-community
#!pip install langchain-openai
#!pip install sentence-transformers
#!pip install ipywidgets tf-keras
#!pip install chromadb langchain-chroma


In [75]:
from langchain_openai import OpenAIEmbeddings

try:
        embeddings = OpenAIEmbeddings( model="text-embedding-3-large")
        result = embeddings.embed_query("Test query")
        print(result)
except Exception as e:
        print(f"Error: {e}")

Error: Connection error.


In [69]:
# 📁 Document Parsing
import os
import fitz  # PyMuPDF
import docx

def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def read_pdf(file_path):
    doc = fitz.open(file_path)
    return "\n".join([page.get_text() for page in doc])

def read_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def parse_documents(folder):
    docs = []
    for root, _, files in os.walk(folder):
        for file in files:
            path = os.path.join(root, file)
            ext = file.lower().split('.')[-1]
            try:
                if ext == 'txt':
                    text = read_txt(path)
                elif ext == 'pdf':
                    text = read_pdf(path)
                elif ext == 'docx':
                    text = read_docx(path)
                else:
                    continue
                docs.append({'text': text, 'path': path})
            except Exception as e:
                print(f"Failed to read {file}: {e}")
    return docs



In [ ]:
# 🧠 Embedding Generation and Storage with ChromaDB
from langchain_community.vectorstores import Chroma

from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document


# Load environment variables
import os
from dotenv import load_dotenv

# Load the .env file
# Set the API key globally
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
OpenAIEmbeddings.api_key = OPENAI_API_KEY


def store_embeddings(docs, persist_directory="VectorDB"):

    vector_store = Chroma(
    collection_name="my_collection",
    embedding_function=OpenAIEmbeddings(),
    persist_directory=persist_directory,
    )

    #create an list of texts 
    texts = [doc['text'] for doc in docs]
    
    #add path to the metadata
    metadatas = [{"source": doc['path']} for doc in docs]

    # Generate unique IDs for each document
    ids = [f"doc_{i}" for i in range(len(texts))]

    # can create the embeddings using openAI or Chromedb hugginFace
    embeddings = OpenAIEmbeddings().embed_documents(texts)
    #embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    # Add documents with precomputed embeddings
    vector_store._collection.upsert(
        documents=texts,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids,
    )
    
    return vector_store

In [71]:
# 🔍 Querying with LangChain
def query_db(query, persist_directory="VectorDB"):

    #chat gpt model
    #embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    results = vectordb.similarity_search(query)
    return results

# Main Program

In [72]:
# Parse documents from a folder
docs = parse_documents("Docs")
# Store them in vector DB
store_embeddings(docs)
# Query the assistant
results = query_db("Maitenance in general")
for r in results:
    print(r.metadata["source"], "\n", r.page_content[:200], "...\n")

C:\Users\PintoMEXT\AppData\Local\Temp\ipykernel_27732\3654894931.py:22: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


APIConnectionError: Connection error.